In [0]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/default/pokemon/pokemon_dataset.xls")
df.display()
df.write.mode("overwrite").parquet("/Volumes/workspace/default/pokemon/bronze/pokemon_dataset_full.parquet")

Criação da Fato Pokemon, buscando o raw data e trazendo o pokemon e seus status

In [0]:
from pyspark.sql.functions import col
df_dim_pokemon = df.select(col("id").alias("id"), col("name").alias("name"), col("base_experience").alias("base_experience"), col("height").alias("height"), col("weight").alias("weight"),col("attack").alias("attack"),col("defense").alias("defense"),col("hp").alias("hp"),col("special-attack").alias("special_attack"),col("special-defense").alias("special_defense"),col("speed").alias("speed"))

df_dim_pokemon.display()

df.write.mode("overwrite").parquet("/Volumes/workspace/default/pokemon/gold/pokemon_ft_stats.parquet")

Tratamento Abilities

In [0]:
from pyspark.sql.functions import col, explode, trim, regexp_replace, monotonically_increasing_id, split

# Corrige o formato da string para algo que o Spark possa dividir corretamente
df_split = df.withColumn(
    "ability_clean",
    regexp_replace(col("abilities"), r"[^\w,]", "")  # remove qualquer coisa que não seja letra, número ou vírgula
).withColumn(
    "ability_array",
    split(col("ability_clean"), ",")  # divide por vírgula
)

df_split.select("id", "name", "abilities", "ability_array").display()

df.write.mode("overwrite").parquet("/Volumes/workspace/default/pokemon/silver/pokemon_dim_ability.parquet")


Criação da DIM_ability

In [0]:
from pyspark.sql.functions import size, col, expr

max_abilities = df_split.select(
    size(col("ability_array")).alias("qtd")
).agg(
    {"qtd": "max"}
).collect()[0][0]

print(f"Número máximo de habilidades: {max_abilities}")

df_habilidades = df_split.select(
    col("id").alias("pokemon_id"),
    *[
        expr(f"try_element_at(ability_array, {i+1})").alias(f"habilidade_{i+1}")
        for i in range(max_abilities)
    ]
)

display(df_habilidades)
df.write.mode("overwrite").parquet("/Volumes/workspace/default/pokemon/gold/pokemon_dim_abilities.parquet")

Tratamento DIM_Type

In [0]:
from pyspark.sql.functions import col
df_dim_types = df.select(col("id").alias("id"),col("types").alias("types"))

df_dim_types.display()

In [0]:
df_dim_types = df.withColumn(
    "types_clean",
    regexp_replace(col("types"), r"[^\w,]", "")  # remove qualquer coisa que não seja letra, número ou vírgula
).withColumn(
    "types_array",
    split(col("types_clean"), ",")  # divide por vírgula
)

df_dim_types.select("id", "types", "types_array").display()

df.write.mode("overwrite").parquet("/Volumes/workspace/default/pokemon/silver/pokemon_dim_type.parquet")


In [0]:
from pyspark.sql.functions import size, col, expr

max_types = df_dim_types.select(
    size(col("types_array")).alias("qtd")
).agg(
    {"qtd": "max"}
).collect()[0][0]

print(f"Número máximo de tipos: {max_types}")

df_dim_types = df_dim_types.select(
    col("id").alias("id"),
    *[
        expr(f"try_element_at(types_array, {i+1})").alias(f"tipo_{i+1}")
        for i in range(max_types)
    ]
)

display(df_dim_types)

df.write.mode("overwrite").parquet("/Volumes/workspace/default/pokemon/gold/pokemon_dim_type.parquet")